<a href="https://colab.research.google.com/github/nikhilongo/Spam-ham-classification/blob/main/spam_ham_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
df = pd.read_csv("/content/SMSSpamCollection.txt", sep = '\t', names = ['label','message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
y = pd.get_dummies(df['label'],drop_first=True)

In [4]:
y.value_counts()

,count
spam,
False,4825
True,747


In [11]:
y.shape

(5572,)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
corpus = []
import re
for i in df['message']:
  review = re.sub('[^a-zA-Z]',' ',i)
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [8]:
corpus[0:3]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary = True)
x = cv.fit_transform(corpus).toarray()

In [60]:
len(cv.vocabulary_)

6296

In [12]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,train_size=0.8)

In [13]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
model1.fit(xts,yts)
pred1 = model1.predict(xts)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
accuracy_score(pred1,yts)

0.9946188340807175

In [15]:
print(classification_report(yts,pred1))

              precision    recall  f1-score   support

       False       0.99      1.00      1.00       957
        True       1.00      0.96      0.98       158

    accuracy                           0.99      1115
   macro avg       1.00      0.98      0.99      1115
weighted avg       0.99      0.99      0.99      1115



In [17]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(xtr,ytr)
pred3 = rf_model.predict(xts)
print("accuracy of rf is: ",accuracy_score(pred3,yts))
print(classification_report(pred3,yts))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


accuracy of rf is:  0.97847533632287
              precision    recall  f1-score   support

       False       1.00      0.98      0.99       979
        True       0.85      0.99      0.92       136

    accuracy                           0.98      1115
   macro avg       0.93      0.98      0.95      1115
weighted avg       0.98      0.98      0.98      1115



#Using tfidf

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(2,3))
x = tf.fit_transform(corpus).toarray()
xtr,xts,ytr,yts = train_test_split(x,y,train_size=0.8)

In [48]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB()
model2.fit(xtr,ytr)
pred2 = model2.predict(xts)
print(accuracy_score(yts,pred2))
print(classification_report(yts,pred2))

0.9730941704035875
              precision    recall  f1-score   support

         ham       0.99      0.98      0.98       962
        spam       0.87      0.94      0.91       153

    accuracy                           0.97      1115
   macro avg       0.93      0.96      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [19]:
rf_model1 = RandomForestClassifier()
rf_model1.fit(xtr,ytr)
pred4 = rf_model1.predict(xts)
print("accuracy of rf is: ",accuracy_score(pred4,yts))
print(classification_report(pred4,yts))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


accuracy of rf is:  0.9614349775784753
              precision    recall  f1-score   support

       False       1.00      0.96      0.98      1016
        True       0.70      1.00      0.82        99

    accuracy                           0.96      1115
   macro avg       0.85      0.98      0.90      1115
weighted avg       0.97      0.96      0.96      1115



#Embedding

In [57]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,LSTM,Bidirectional

In [61]:
voc_size = 7000
ohe = [one_hot(sent,voc_size) for sent in corpus]
ohe[0:3]

[[5613,
  1293,
  6829,
  6254,
  3532,
  2620,
  4155,
  614,
  288,
  40,
  5685,
  2079,
  1039,
  3855,
  4307,
  1745],
 [2155, 2327, 4564, 2208, 2652, 2727],
 [3586,
  3032,
  1952,
  5675,
  1251,
  6656,
  3563,
  183,
  2914,
  5781,
  1217,
  3577,
  6656,
  2422,
  3032,
  5577,
  939,
  639,
  2952,
  2325,
  2063]]

In [62]:
max = 0
for i in ohe:
  if len(i) > max:
    max = len(i)
print(max)

77


In [64]:
padding = pad_sequences(ohe,maxlen=max,padding='pre')
padding[34]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 1989, 6109,
       1159, 1404,  406, 1020, 2084, 6938, 4264, 6828, 6583, 6828, 1020],
      dtype=int32)

In [65]:
dim = 20
model = Sequential()
model.add(Embedding(voc_size,dim,input_length=max))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [68]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [74]:
xtrain,xtest,ytrain,ytest = train_test_split(padding,y,train_size=0.8)
model.fit(xtrain,ytrain,epochs=10,validation_data=(xtest,ytest))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.8827 - loss: 0.3415 - val_accuracy: 0.9758 - val_loss: 0.0843
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.9792 - loss: 0.0734 - val_accuracy: 0.9821 - val_loss: 0.0575
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9880 - loss: 0.0384 - val_accuracy: 0.9821 - val_loss: 0.0579
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.9948 - loss: 0.0161 - val_accuracy: 0.9839 - val_loss: 0.0489
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.9979 - loss: 0.0072 - val_accuracy: 0.9821 - val_loss: 0.0537
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.9980 - loss: 0.0107 - val_accuracy: 0.9874 - val_loss: 0.0563
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9990 - loss: 0.0036 - val_accuracy: 0.9848 - val_loss: 0.0577
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9978 - loss: 0.0100 - val_a

In [75]:
model1 = Sequential()
model1.add(Embedding(voc_size,dim,input_length=max))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [76]:
model1.fit(xtrain,ytrain,epochs=10,validation_data=(xtest,ytest))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 112ms/step - accuracy: 0.8561 - loss: 0.3622 - val_accuracy: 0.9740 - val_loss: 0.0796
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 109ms/step - accuracy: 0.9794 - loss: 0.0616 - val_accuracy: 0.9812 - val_loss: 0.0574
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 111ms/step - accuracy: 0.9905 - loss: 0.0272 - val_accuracy: 0.9830 - val_loss: 0.0535
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 15s 110ms/step - accuracy: 0.9951 - loss: 0.0190 - val_accuracy: 0.9839 - val_loss: 0.0583
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 109ms/step - accuracy: 0.9987 - loss: 0.0072 - val_accuracy: 0.9865 - val_loss: 0.0495
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 109ms/step - accuracy: 0.9984 - loss: 0.0071 - val_accuracy: 0.9874 - val_loss: 0.0583
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 111ms/step - accuracy: 0.9989 - loss: 0.0038 - val_accuracy: 0.9883 - val_loss: 0.0649
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 108ms/step - accuracy: 0.9990 - loss: 0